In [1]:
from IPython.core.display import HTML
from IPython.display import Image, display
display(HTML("<style>.container { width:100% !important; }</style>"))
import config
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from imageio import imwrite
import yaml
from fastai.core import Path
from fastai.vision import load_learner, defaults
import torch as torch
%matplotlib inline
%reload_ext autoreload
%autoreload 2
defaults.device = torch.device('cpu')


from tagging_tool_utils import *
from image_graph import *
import ipywidgets as widgets
letter_mappings = yaml.load(open('../yaml/letter_maps.yaml'))

In [2]:
pic_name = 'GK_RDR_PG3_2'

In [3]:
gr_path = Path('../greek_pages/page_graphs/')
im_path = Path('../greek_pages/page_images/')
gname = pic_name + '.txt'
im_name = pic_name + '.jpeg'

model_name = 'rn_34.pkl'
config.model = load_learner('../models/', model_name) # classifier for greek and english characters, this model is on my local machine
letter_path = Path('../all_data/lgi_data') # location for data, on local machine
config.letter_dest = letter_path/'pg3_data'
get_letter_dict()

config.img_arr = get_image_array(im_path/im_name)

In [6]:
yaml_pth = config.letter_dest/'yaml'
with open(yaml_pth/'new_components.yaml', 'rb') as f:
    new_components = yaml.load(f)
config.components = new_components
config.itr = 0
config.temp_path = yaml_pth
config.rows, config.cols = config.img_arr.shape
config.letter_mappings = letter_mappings

In [7]:
lb, ub, lbr, ubr = get_bounds(config.itr)

In [8]:
imwrite(config.temp_path/'temp_letter.jpg', config.img_arr[lbr:ubr,lb:ub])
file = open(config.temp_path/"temp_letter.jpg", "rb")
image = file.read()
config.image = widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)

In [ ]:
letter_button = widgets.Button(description='Update Letter')
skip_button = widgets.Button(description='Skip')
context_button = widgets.Button(description='Get Context')
reset_context_button = widgets.Button(description='Reset Context')
config.output = widgets.Output()
config.letter_options = widgets.Dropdown(options=[''])
config.letter_options.options =  [''] + get_top_preds(config.temp_path/"temp_letter.jpg")
config.textbox = widgets.Text()
    
letter_button.on_click(update_letter)
skip_button.on_click(skip)
context_button.on_click(get_context)
reset_context_button.on_click(reset_context)
# display
widgets.VBox([config.image,config.textbox,config.letter_options,letter_button,skip_button,context_button,reset_context_button,config.output])

Jupyter won't load the widget so here is an image of what it looks like:
<img src="../imgs/widget_img.png" style="width: 700px; height=50px;" />

The widget displays the character image, and at the bottom prints out the top 3 predictions along with their probabilities. The first input box is a text input for typing the correct label. The second is a dropdown with the top 3 predictions. There is an option to skip, although there are no scenarios where you would do this in practice, just for convenience. Get Context expands the images so you can see the surrounding characters, and Reset Context returns the context back to just the character image

In [ ]:
yaml.dump(letter_mappings,open('nbs/letter_maps.yaml','w'))